In [34]:
import requests
import warnings 
from urllib.request import urlopen
import time 
from bs4 import BeautifulSoup
import re
from  tqdm import tqdm
from datetime import datetime
import re
import pandas as pd

# 1

## 1.1. Get the list of places

In [2]:
exception=[]  #In this list we store the exception in order to understand what is wrong.

#Since the link of the page contains number we use a for loop in order to change that number and so change the links.


for i in tqdm(range(1,401)):
    url='https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count' #Move to the next web page 
    risposta=requests.get(url)                                                 #Request https protocolo        
    testo=BeautifulSoup(risposta.text,'html.parser')                           #Format the html
    
    try:
        first_div=testo.find_all("div",{"class":"row vue-js-bte-place-parent js-vue-component-wrap"}) #find the tag
        
        
        for j in range(0,len(first_div)):
            try:
                second_div=first_div[j].find_all("div",{"class":"col-md-4 col-sm-6 col-xs-12"})
    
                for k in range(0,len(second_div)):
                    try:
                        a=second_div[k].find("a").get("href").strip()    #extract the link of the page
                     
                        with open('links.txt',"a")as f:
                            f.write("https://www.atlasobscura.com"+str(a))  #write down on a txt file the complete link of the page.
                            f.write('\n')
                    except:
                        exception.append([i,j,k,risposta])      
            except:
                exception.append([i,j,None,risposta])   
    except:
        exception.append([i,None,None,risposta])
        
    
    time.sleep(1) #In ordere to avoid "429 Too Many Requests"
       
            
        

  0%|          | 1/400 [00:02<14:25,  2.17s/it]


KeyboardInterrupt: 

## 1.2. Crawl places

In [29]:
file = open('links.txt')
content = file.readlines()
for i in tqdm(range(0,len(content))):
    url=content[i].strip()    
    risposta=requests.get(url)                                               
    with open('HTML/html'+str(i)+'.txt',"a")as f:
        f.write(risposta.text)  #write down on a txt file the complete link of the page.
    time.sleep(1)


100%|██████████| 7200/7200 [2:18:44<00:00,  1.16s/it]  


## 1.3 Parse downloaded pages

### Function that extract data from single HTML

In [48]:
def estrazione(testo):  
    errors=[]
    #1 Place Name (to save as placeName): String.
    try:
        placeName=testo.find("h1",{"class":"DDPage__header-title"}).text.strip()
    except:
        placeName=" "
        errors.append(1)
    #---#
    #2 Place Tags (to save as placeTags): List of Strings.
    try:
        placeTags=[]
        ff2=testo.find("div",{"class":"item-tags col-xs-12"})
        ff2=ff2.find_all("span")
        for i in range(len(ff2)):
            placeTags.append(ff2[i].find("a").text.strip())
    except:
        placeTags=[" "]
        errors.append(2)



    #---#
    conta=testo.find_all("div",{"class","title-md item-action-count"})

    #3 # of people who have been there (to save as numPeopleVisited): Integer.
    try:
        numPeopleVisited=conta[0].text.strip()
    except:
        numPeopleVisited=" "
        errors.append(3)
    #---#
    #4 # of people who want to visit the place(to save as numPeopleWant): Integer.    
    try:
        numPeopleWant=conta[1].text.strip()
    except:
        numPeopleWant=" "
        errors.append(4)
    #---#
    #5 Description (to save as placeDesc): String. Everything from under the first image up to "know before you go" (orange frame on the example image).
    try:
        tag=testo.find("div",{"class":"DDP__body-copy"})
        tag=tag.find_all("p")
        placeDesc=""
        for i in range(0,len(tag)):
            placeDesc += str(tag[i].text)
    except:
        placeDesc=""
        errors.append(5)

    #6 Short Description (to save as placeShortDesc): String. Everything from the title and location up to the image (blue frame on the example image).
    placeShortDesc=""
    try:
        placeShortDesc2=testo.find("h3", {"class","DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc2=" "
        errors.append(6.2)
    placeShortDesc += placeShortDesc2

    #7 Nearby Places (to save as placeNearby): Extract the names of all nearby places, but only keep unique values: List of Strings.
    placeNearby=[]
    try:
        place=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        place=place.find("div",{"class":"DDPageSiderail"})
        place=place.find("div", {"class":"DDPageSiderailRecirc"})
        place=place.find_all("a")
        for i in range(0,len(place)):
            luogo=place[i].find("div",{"class":"DDPageSiderailRecirc__item-title"})
            placeNearby.append(luogo.text.strip())
    except:
        placeNearby=[" "]
        errors.append(7)
    #8 Address of the place(to save as placeAddress): String.
    try:
        placeAddress=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        placeAddress=placeAddress.find("div",{"class":"DDPageSiderail"})
        placeAddress=placeAddress.find("aside", {"class":"DDPageSiderail__details"})
        placeAddress=placeAddress.find("address",{"class":"DDPageSiderail__address"})
        placeAddress=placeAddress.find("div").get_text(",").strip().rstrip(',').strip()
    except:
        placeAddress=" "
        errors.append(8)

    #9 Altitude and Longitude of the place's location(to save as placeAlt and placeLong): Integers
    try:
        coordinate=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail"})
        coordinate=coordinate.find("aside", {"class":"DDPageSiderail__details"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).text.strip().split()

        placeAlt=float(coordinate[0].replace(",",""))
        placeLong=float(coordinate[1].replace(",",""))
    except:
        placeAlt=" "
        placeLong=" "
        errors.append(9)

    #10 The username of the post editors (to save as placeEditors): List of Strings
    placeEditors=[]
    createdby=[]
    Editors=testo.find("div",{"id":"ugc-module"})
    Editors=Editors.find("div",{"class":"DDPContributors"})
    Editors=Editors.find_all("div",{"class":"ugc-editors"})

    try:
        for i in range(0,len(Editors)):
            tipo=Editors[i].find("h6").text.strip()
            if "edited by" == tipo.lower():
                try:
                    g=Editors[i].find("div",{"class":"DDPContributorsList"})
                    g=g.find("div",{"class":"js-editor-list hidden"})
                    g=g.find_all("li")
                    for  i in range(0,len(g)):
                        a=g[i].find("a").find("span").text.strip()
                        placeEditors.append(a)
                except:
                    placeEditors=[" "]
                    #errors.append(10.1)
                    
            elif "added by"==tipo.lower():
                try:
                    creator=Editors[i].find("div",{"class":"DDPContributorsList"})
                    createdby.append(creator.find("a").text)
                except:
                    createdby=[" "]
                   # errors.append(10.2)
    except:
        createdby=[" "]
        placeEditors=[" "] 
        errors.append(10)
    placeEditors=",".join(placeEditors)
    createdby=",".join(createdby)


    #11 Post publishing date (to save as placePubDate): datetime.
    try:
        placePubDate=testo.find("div",{"id":"ugc-module"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributors"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributor__name"}).text.strip()
        placePubDate=datetime.strptime(placePubDate, "%B %d, %Y").date() #year, month, day
    except:
        placePubDate=" "
        errors.append(11)

    #12 The names of the lists that the place was included in (to save as placeRelatedLists): List of Strings.
    placeRelatedLists=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[2].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedLists.append(m)
    except:
        placeRelatedLists=[" "]
        errors.append(12)

    #13 The names of the related places (to save as placeRelatedPlaces): List of Strings.
    placeRelatedPlaces=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[1].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedPlaces.append(m)
    except:
        placeRelatedPlaces=[" "]
        errors.append(13)

    #14 The URL of the page of the place (to save as placeURL):String
    try:
        placeURL=testo.find("link",{"rel":"canonical"}).get("href")
    except:
        placeURL=" "
        errors.append(14)
    
    

    try:
        link=testo.find("div",{"class":"DDPage__header-place-location"})
        link=link.find("a").get("href").strip()
    except:
        link=" "
       
    
    d={"placeName":placeName,"placeTags": ",".join(placeTags),"numPeopleVisited":int(numPeopleVisited),"numPeopleWant":int(numPeopleWant),"placeDesc":placeDesc,"placeShortDesc":placeShortDesc,
    "placeNearby":",".join(placeNearby),"placeAddress":placeAddress.strip(),"placeAlt":placeAlt,"placeLong":placeLong,"createdby":createdby,"placeEditors":placeEditors,"placePubDate":placePubDate,
    "placeRelatedLists":",".join(placeRelatedLists),"placeRelatedPlaces":",".join(placeRelatedPlaces),"placeURL":placeURL}

    return  d,errors      

## Create dataframe

#### Create 7200 tsv files, place_i.tsv and one csv file

In [49]:
anomalie={}
for i in tqdm(range(0,7200)):
    
    with open('HTML/html'+str(i)+'.txt') as f:
        testo = f.read()

    testo=BeautifulSoup(testo,'html.parser') 
    d,errori=estrazione(testo)
    
    if len(errori) != 0:
        anomalie[str(i)]=errori
        
    df=pd.DataFrame(d,index=[0])
    df.to_csv("TSV/place_"+str(i)+".tsv",sep="\t")
    if i==0 :
      dfff=pd.DataFrame(d,index=[i])
    else:
      dataframe=pd.DataFrame(d,index=[i])
      dfff=dfff.append(dataframe)
        
dfff.to_csv("places2.csv",sep="\t")

100%|██████████| 7200/7200 [22:56<00:00,  5.23it/s]


#### Overview of the dataset

In [39]:
df=pd.read_csv("places2.csv",delimiter="\t")
df.shape

(7200, 17)

#### Analysis of the errors

In [41]:
with open('anomalie.txt',"w")as f:
    for i in anomalie.keys():
        f.write(i)
        f.write(str(anomalie[i]))
        f.write("\n")

In [40]:
df.placeURL[df.index==6037]

6037    https://www.atlasobscura.com/places/shangri-la-botanical-gardens-nature-center
Name: placeURL, dtype: object

In [43]:
for i in anomalie.keys():
    print(anomalie[i],i)

[12] 79
[12] 84
[12] 91
[12] 92
[12] 100
[12] 102
[12] 107
[12] 109
[12] 113
[12] 115
[12] 120
[12] 124
[12] 133
[12] 134
[12] 135
[12] 136
[12] 141
[12] 144
[12] 145
[12] 149
[12] 151
[12] 152
[12] 154
[12] 159
[12] 161
[12] 162
[12] 167
[12] 172
[12] 174
[12] 176
[12] 181
[12] 184
[12] 189
[12] 190
[12] 198
[12] 204
[12] 205
[12] 206
[12] 207
[12] 208
[12] 210
[12] 221
[12] 222
[12] 223
[12] 226
[12] 228
[12] 230
[12] 231
[12] 232
[12] 234
[12] 236
[12] 239
[12] 245
[12] 248
[12] 250
[12] 251
[12] 256
[12] 260
[12] 261
[12] 263
[12] 264
[12] 269
[12] 272
[12] 273
[12] 274
[12] 279
[12] 282
[12] 285
[12] 286
[12] 287
[12] 290
[12] 294
[12] 296
[12] 297
[12] 299
[12] 300
[12] 301
[12] 302
[12] 304
[12] 308
[12] 309
[12] 311
[12] 312
[12] 313
[12] 315
[12] 317
[12] 326
[12] 327
[12] 328
[12] 329
[12] 333
[12] 334
[12] 335
[12] 338
[12] 342
[12] 343
[12] 344
[12] 346
[12] 352
[12] 353
[12] 354
[12] 357
[12] 360
[12] 361
[12] 362
[12] 365
[12] 366
[12] 367
[12] 372
[12] 375
[12] 377
[12] 